# Importing Libraries

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from z3 import *
from sklearn.cluster import DBSCAN
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import sys
import warnings
warnings.filterwarnings("ignore")
import json

# FIXED PARAMETERS

In [23]:
NUM_ZONES = 5

# Clustering

In [24]:
# returns dbscan clusters
def dbscan(X, eps, min_samples):
    db = DBSCAN(eps=eps, min_samples=min_samples)
    cluster = db.fit(X)
    #y_pred = db.fit_predict(X)
    #plt.scatter(X[:,0], X[:,1],c=y_pred, cmap='Paired')
    #plt.title("DBSCAN")
    #print("noise", sum(cluster.labels_ ==-1), "regular", sum(cluster.labels_ !=-1))
    #print(np.unique(cluster.labels_))
    return cluster

# Convex Hull

In [25]:
# returns convex hull of points associated with zones
def convex_hull(zone, points):
        '''
        Parameters
        ----------
        points : Array
            A set of points.
            
        Returns
        -------
        vertices : Points
            Arranges convex hull vertices in counter-clockwise orientation.
        '''    
        
        hull = ConvexHull(points)
        
        
        #plt.plot(points[:,0], points[:,1], 'o')
        #plt.ylim(0,120)
        #for simplex in hull.simplices:
        #    plt.plot(points[simplex, 0], points[simplex, 1], 'k-')
            
        simplices = hull.simplices
        
        vertices = []
        for index in hull.vertices:
            vertices.append((points[index][0], zone, points[index][1]))
        vertices.append((points[hull.vertices[0]][0], zone, points[hull.vertices[0]][1]))
        
        return vertices

# Cluster Boundary Acquisition

In [29]:
def get_cluster(dataframe, eps, min_samples):
    list_cluster = []
    for zone in range(NUM_ZONES):
        #plt.figure()
        mod_dataframe = pd.DataFrame()

        for i in range(len(dataframe)):
            if int(dataframe['Occupant\'s Zone'][i] == zone):
                mod_dataframe = mod_dataframe.append(dataframe.loc[i, ['Zone Arrival Time (Minute)', 'Stay Duration (Minute)']])

        features = np.empty([len(mod_dataframe), 2])
        features[:, 0:1] = mod_dataframe.loc[:, ['Zone Arrival Time (Minute)']].values
        features[:, 1:] = mod_dataframe.loc[:, ['Stay Duration (Minute)']].values
        cluster_model = dbscan(features, eps, min_samples)
        labels = cluster_model.labels_
        
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        for cluster in range(n_clusters):
            points = []
            for k in range(len(labels)):
                if labels[k] == cluster:
                    points.append(features[k])
            points = np.array(points)

            if len(points) >= 3:
                try:
                    vertices = convex_hull(zone, points)
                except:
                    print("Exception")
            else:
                print("zone", zone, "Problem Found!")

            list_cluster.append({"zone_id": zone, "cluster_id": cluster, "points": vertices})

    return list_cluster

# Clustering for House-A and Occupant-1

In [30]:
cleaned_duration_dataframe_house_A_occ_1 = pd.read_csv("../../data/cleaned/Cleaned-Dataframe_House-A_Occupant-1.csv")
cleaned_duration_dataframe_house_A_occ_1
clusters_house_A_occ_1 = get_cluster(cleaned_duration_dataframe_house_A_occ_1, 30, 3)
# saving clusters to file
with open('../../data/shatter/DBSCAN_Clusters-House_A_Occ-1.txt', 'w') as f:
    f.write(json.dumps(clusters_house_A_occ_1))

Exception


# Clustering for House-A and Occupant-2

In [31]:
cleaned_duration_dataframe_house_A_occ_2 = pd.read_csv("../../data/cleaned/Cleaned-Dataframe_House-A_Occupant-2.csv")
cleaned_duration_dataframe_house_A_occ_2
clusters_house_A_occ_2 = get_cluster(cleaned_duration_dataframe_house_A_occ_2, 26, 3)
# saving clusters to file
with open('../../data/shatter/DBSCAN_Clusters-House_A_Occ-2.txt', 'w') as f:
    f.write(json.dumps(clusters_house_A_occ_2))

# Clustering for House-B and Occupant-1

In [32]:
cleaned_duration_dataframe_house_B_occ_1 = pd.read_csv("../../data/cleaned/Cleaned-Dataframe_House-B_Occupant-1.csv")
cleaned_duration_dataframe_house_B_occ_1
clusters_house_B_occ_1 = get_cluster(cleaned_duration_dataframe_house_B_occ_1, 30, 3)
# saving clusters to file
with open('../../data/shatter/DBSCAN_Clusters-House_B_Occ-1.txt', 'w') as f:
    f.write(json.dumps(clusters_house_B_occ_1))

# Clustering for House-B and Occupant-2

In [33]:
cleaned_duration_dataframe_house_B_occ_2 = pd.read_csv("../../data/cleaned/Cleaned-Dataframe_House-B_Occupant-2.csv")
cleaned_duration_dataframe_house_B_occ_2
clusters_house_B_occ_2 = get_cluster(cleaned_duration_dataframe_house_B_occ_2, 30, 3)
# saving clusters to file
with open('../../data/shatter/DBSCAN_Clusters-House_B_Occ-2.txt', 'w') as f:
    f.write(json.dumps(clusters_house_B_occ_2))

Exception
Exception
Exception


# Range Claculation Function

In [10]:
def is_left( x, y, init_x, init_y, final_x, final_y):
     return ((final_x - init_x)*(y - init_y) - (final_y - init_y)*(x - init_x)) >= 0

def range_calculation(list_cluster):
    num_timeslots = 1440
    list_time_min = [[[] for j in range(num_timeslots)] for i in range(NUM_ZONES + 1)]
    list_time_max = [[[] for j in range(num_timeslots)] for i in range(NUM_ZONES + 1)]

    for i in range(len(list_cluster)):
        zone_id = list_cluster[i]["zone_id"]
        min_x_range = 1440
        max_x_range = 0

        ##################################################################
        ##################### Zone Constraints ###########################
        ##################################################################
        x = Int('x')
        y = Int('y')

        points = list_cluster[i]["points"]
        zone_constraints = []

        and_constraints = []
        for j in range(len(points) - 1):
            and_constraints.append(is_left(x, y, points[j][0], points[j][2], points[j + 1][0], points[j + 1][2]))

        zone_constraints.append(And(and_constraints))

        #print(zone_constraints)
        ####### Minimum value of X range #######
        o = Optimize()
        o.add(zone_constraints)
        o.minimize(x)
        o.check()
        
        min_x_range = int(str(o.model()[x]))

        ####### Maximum value of X range #######
        o = Optimize()
        o.add(zone_constraints)
        o.maximize(x)
        o.check()
        #print(o.model()[x])

        max_x_range = int(str(o.model()[x]))
        
        
        for j in range(min_x_range, max_x_range):
            ####### Minimum value of Y range #######
            o = Optimize()
            o.add(zone_constraints)
            o.add(x == j)
            o.minimize(y)
            o.check()

            min_y_range = o.model()[y]
            if min_y_range == None:
                min_y_range = 0

            ####### Maximum value of Y range #######
            o = Optimize()
            o.add(zone_constraints)
            o.add(x == j)
            o.maximize(y)
            o.check()

            max_y_range = o.model()[y]
            if max_y_range == None:
                max_y_range = 0
                
            list_time_min[zone_id][j].append(int(str(min_y_range)))
            list_time_max[zone_id][j].append(int(str(max_y_range)))
    return list_time_min, list_time_max

# Attack Scheduling Function

In [11]:
def attack_scheduling(list_cluster, num_timeslots, start_time, final_time):
    list_time_min, list_time_max = range_calculation(list_cluster)
    num_timeslots = interval = 10

    prev_stay = 1
    prev_schedule = -1
    ultimate_cost = 0
    final_schedule = schedule = np.zeros((final_time - start_time))

    for init_time in range(start_time, final_time, interval):
        cost = [Int( 'cost_' + str(i)) for i in range(NUM_ZONES)]
        zones = [Int( 'zones_' + str(i)) for i in range(NUM_ZONES)]

        schedule = [Int( 'schedule_' + str(i)) for i in range(init_time, init_time + interval)]
        stay = [Int( 'stay_' + str(i)) for i in range(num_timeslots)]
        slot_cost = [Int( 'slot_cost_' + str(i)) for i in range(num_timeslots)]

        total_cost = Int('total_cost')

        o = Optimize()
        o.add(cost[0] == 0)
        o.add(cost[1] == 1)
        o.add(cost[2] == 2)
        o.add(cost[3] == 4)
        o.add(cost[4] == 3)


        ############################################################################
        ################## schedule should be withing a valid zone #################
        ############################################################################
        for t in range(len(schedule)):
            or_constraints = []
            for z in range(NUM_ZONES):
                or_constraints.append(schedule[t] == z)
            o.add(Or(or_constraints))

        ###############################################################################################################
        ################## if zone stay threshdold in current time is 0, do not schedule to that zone #################
        ###############################################################################################################

        # base case
        for z in range(1, NUM_ZONES):
            if list_time_min[z][init_time] == [] or (len(list_time_max[z][init_time]) == 1 and list_time_max[z][init_time][0] == 0):
                o.add(Implies(schedule[0] != prev_schedule, schedule[t] != z))

        for t in range(1, len(schedule)):
            for z in range(1, NUM_ZONES):
                if list_time_min[z][init_time + t] == [] or (len(list_time_max[z][init_time + t]) == 1 and list_time_max[z][init_time + t][0] == 0):
                    o.add(Implies(schedule[t] != schedule[t - 1], schedule[t] != z))

        #######################################################################################################################
        ############################################ constraints of stay ######################################################
        #######################################################################################################################
        ######## base case for time 0 ############
        if init_time == 0:
            o.add(stay[0] == 1)
        else:
            o.add(Implies(schedule[0] == prev_schedule, stay[0] == prev_stay + 1))
            o.add(Implies(Not(schedule[0] == prev_schedule), stay[0] == 1))

        for t in range(1, len(schedule)):
            continue_staying = (schedule[t] == schedule[t - 1])
            increment_stay = (stay[t] == stay[t - 1] + 1)
            reset_stay = (stay[t] == 1)

            o.add(Implies(continue_staying, increment_stay))
            o.add(Implies(Not(continue_staying), reset_stay))

        #######################################################################################################################
        ############ move to a zone different that previous timeslot if stay > max threshold in previous timeslot #############
        #######################################################################################################################
        ######## base case for time 0 ############
        o.add(Implies(prev_stay == max(list_time_max[prev_schedule][init_time - prev_stay], default=0), schedule[0] != prev_schedule))

        for t in range(1, len(schedule)):
            max_stay_threshold = 0
            for z in range(1, NUM_ZONES):
                for p_t in range(1, init_time + len(schedule)):
                    continue_staying = (schedule[t] == schedule[t - 1])
                    o.add(Implies(And(schedule[t - 1] == z, stay[t - 1] == p_t, p_t == max(list_time_max[z][init_time + t - p_t], default=0)), Not(continue_staying)))

        #######################################################################################################################
        # must stay in the zone same as the previous timeslot if stay < max && stay is in previous timeslot is out of cluster #
        #######################################################################################################################
        ######## base case for time 0 ############
        ranges_stay_constraints = []
        for k in range(len(list_time_min[prev_schedule][init_time - prev_stay])):
            ranges_stay_constraints.append(And(prev_stay >= list_time_min[prev_schedule][init_time - prev_stay][k], prev_stay <= list_time_max[prev_schedule][init_time - prev_stay][k]))            

        if init_time != 0:
            o.add(Implies(Not(Or(ranges_stay_constraints)), schedule[0] == prev_schedule))

        for t in range(1, len(schedule)):
            for z in range(1, NUM_ZONES):
                for p_t in range(1, init_time + t + 1):
                    ranges_stay_constraints = []
                    for k in range(len(list_time_min[z][init_time + t - p_t])):
                        ranges_stay_constraints.append(And(p_t >= list_time_min[z][init_time + t - p_t][k], p_t <= list_time_max[z][init_time + t - p_t][k]))            

                    continue_staying = (schedule[t] == schedule[t - 1])
                    o.add(Implies(And(schedule[t - 1] == z, stay[t - 1] == p_t, Not(Or(ranges_stay_constraints))), continue_staying))

        for t in range(len(schedule)):
            for z in range(NUM_ZONES):
                o.add(Implies(schedule[t] == z, slot_cost[t] == cost[z]))

        o.add(total_cost == Sum(slot_cost))

        #o.add(schedule[0] == 3)

        o.maximize(total_cost)

        print(o.check())
        print(init_time, o.model()[total_cost])
        ultimate_cost += int(str(o.model()[total_cost]))

        print("Ulimate Cost", init_time, ultimate_cost)

        print("Num_timeslot", num_timeslots, "Optimal_cost", o.model()[total_cost])

        for t in range(num_timeslots):
            print("schedule_" + str(init_time + t), o.model()[schedule[t]])
            final_schedule[init_time + t] = int(str(o.model()[schedule[t]]))

        for t in range(num_timeslots):
            print("stay_" + str(init_time + t), o.model()[stay[t]])

        prev_schedule = int(str(o.model()[schedule[-1]]))
        prev_stay = int(str(o.model()[stay[-1]]))
    return final_schedule

# Greedy Scheduling Function

In [12]:
def greedy_schedule(list_cluster, num_timeslots):
    current_timeslot = 0
    final_timeslot = 0

    alternate_schedule = np.zeros((num_timeslots))
    list_time_min, list_time_max = range_calculation(list_cluster)
    while final_timeslot <= 1439:
        for zone in [3, 4, 2, 1, 0]:
            max_list = list_time_max[zone][current_timeslot]
            for m in range(len(max_list)):
                max_list[m] = int(str(max_list[m]))
            if max_list == []:
                if zone == 0:
                    print("Critical", current_timeslot)
                    final_timeslot = current_timeslot + 1
                    alternate_schedule[current_timeslot: final_timeslot] = 0
                    current_timeslot = final_timeslot
                continue
            max_val = max(max_list)
            if max_val == 0:
                final_timeslot = current_timeslot + 1
                alternate_schedule[current_timeslot: final_timeslot] = 0
                current_timeslot = final_timeslot
            if max_val != 0:
                final_timeslot = current_timeslot + max_val
                print("Here", current_timeslot, final_timeslot)
                alternate_schedule[current_timeslot: final_timeslot] = zone
                print("selected zone", zone)
                current_timeslot = final_timeslot
                break
    return alternate_schedule

# Variable Parameters

In [13]:
num_timeslots = interval = 10
start_time = 0
final_time = 1440

# House-A Occupant-1 Attack Schedule

In [14]:
schedule_house_A_occ_1 = attack_scheduling(clusters_house_A_occ_1, num_timeslots, start_time, final_time)
pd.DataFrame(schedule_house_A_occ_1, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/DBSCAN_House-A_Occ-1_SHATTER_Schedule.csv", index = False)

sat
0 40
Ulimate Cost 0 40
Num_timeslot 10 Optimal_cost 40
schedule_0 3
schedule_1 3
schedule_2 3
schedule_3 3
schedule_4 3
schedule_5 3
schedule_6 3
schedule_7 3
schedule_8 3
schedule_9 3
stay_0 1
stay_1 2
stay_2 3
stay_3 4
stay_4 5
stay_5 6
stay_6 7
stay_7 8
stay_8 9
stay_9 10
sat
10 36
Ulimate Cost 10 76
Num_timeslot 10 Optimal_cost 36
schedule_10 0
schedule_11 3
schedule_12 3
schedule_13 3
schedule_14 3
schedule_15 3
schedule_16 3
schedule_17 3
schedule_18 3
schedule_19 3
stay_10 1
stay_11 1
stay_12 2
stay_13 3
stay_14 4
stay_15 5
stay_16 6
stay_17 7
stay_18 8
stay_19 9
sat
20 20
Ulimate Cost 20 96
Num_timeslot 10 Optimal_cost 20
schedule_20 2
schedule_21 2
schedule_22 2
schedule_23 2
schedule_24 2
schedule_25 2
schedule_26 2
schedule_27 2
schedule_28 2
schedule_29 2
stay_20 1
stay_21 2
stay_22 3
stay_23 4
stay_24 5
stay_25 6
stay_26 7
stay_27 8
stay_28 9
stay_29 10
sat
30 24
Ulimate Cost 30 120
Num_timeslot 10 Optimal_cost 24
schedule_30 2
schedule_31 2
schedule_32 2
schedule_33 2

sat
260 10
Ulimate Cost 260 598
Num_timeslot 10 Optimal_cost 10
schedule_260 1
schedule_261 1
schedule_262 1
schedule_263 1
schedule_264 1
schedule_265 1
schedule_266 1
schedule_267 1
schedule_268 1
schedule_269 1
stay_260 90
stay_261 91
stay_262 92
stay_263 93
stay_264 94
stay_265 95
stay_266 96
stay_267 97
stay_268 98
stay_269 99
sat
270 10
Ulimate Cost 270 608
Num_timeslot 10 Optimal_cost 10
schedule_270 1
schedule_271 1
schedule_272 1
schedule_273 1
schedule_274 1
schedule_275 1
schedule_276 1
schedule_277 1
schedule_278 1
schedule_279 1
stay_270 100
stay_271 101
stay_272 102
stay_273 103
stay_274 104
stay_275 105
stay_276 106
stay_277 107
stay_278 108
stay_279 109
sat
280 10
Ulimate Cost 280 618
Num_timeslot 10 Optimal_cost 10
schedule_280 1
schedule_281 1
schedule_282 1
schedule_283 1
schedule_284 1
schedule_285 1
schedule_286 1
schedule_287 1
schedule_288 1
schedule_289 1
stay_280 110
stay_281 111
stay_282 112
stay_283 113
stay_284 114
stay_285 115
stay_286 116
stay_287 117
stay

sat
500 10
Ulimate Cost 500 838
Num_timeslot 10 Optimal_cost 10
schedule_500 1
schedule_501 1
schedule_502 1
schedule_503 1
schedule_504 1
schedule_505 1
schedule_506 1
schedule_507 1
schedule_508 1
schedule_509 1
stay_500 330
stay_501 331
stay_502 332
stay_503 333
stay_504 334
stay_505 335
stay_506 336
stay_507 337
stay_508 338
stay_509 339
sat
510 10
Ulimate Cost 510 848
Num_timeslot 10 Optimal_cost 10
schedule_510 1
schedule_511 1
schedule_512 1
schedule_513 1
schedule_514 1
schedule_515 1
schedule_516 1
schedule_517 1
schedule_518 1
schedule_519 1
stay_510 340
stay_511 341
stay_512 342
stay_513 343
stay_514 344
stay_515 345
stay_516 346
stay_517 347
stay_518 348
stay_519 349
sat
520 10
Ulimate Cost 520 858
Num_timeslot 10 Optimal_cost 10
schedule_520 1
schedule_521 1
schedule_522 1
schedule_523 1
schedule_524 1
schedule_525 1
schedule_526 1
schedule_527 1
schedule_528 1
schedule_529 1
stay_520 350
stay_521 351
stay_522 352
stay_523 353
stay_524 354
stay_525 355
stay_526 356
stay_52

sat
750 37
Ulimate Cost 750 1535
Num_timeslot 10 Optimal_cost 37
schedule_750 3
schedule_751 3
schedule_752 3
schedule_753 3
schedule_754 3
schedule_755 3
schedule_756 3
schedule_757 4
schedule_758 4
schedule_759 4
stay_750 68
stay_751 69
stay_752 70
stay_753 71
stay_754 72
stay_755 73
stay_756 74
stay_757 1
stay_758 2
stay_759 3
sat
760 39
Ulimate Cost 760 1574
Num_timeslot 10 Optimal_cost 39
schedule_760 4
schedule_761 3
schedule_762 3
schedule_763 3
schedule_764 3
schedule_765 3
schedule_766 3
schedule_767 3
schedule_768 3
schedule_769 3
stay_760 4
stay_761 1
stay_762 2
stay_763 3
stay_764 4
stay_765 5
stay_766 6
stay_767 7
stay_768 8
stay_769 9
sat
770 40
Ulimate Cost 770 1614
Num_timeslot 10 Optimal_cost 40
schedule_770 3
schedule_771 3
schedule_772 3
schedule_773 3
schedule_774 3
schedule_775 3
schedule_776 3
schedule_777 3
schedule_778 3
schedule_779 3
stay_770 10
stay_771 11
stay_772 12
stay_773 13
stay_774 14
stay_775 15
stay_776 16
stay_777 17
stay_778 18
stay_779 19
sat
780 

sat
1000 40
Ulimate Cost 1000 2450
Num_timeslot 10 Optimal_cost 40
schedule_1000 3
schedule_1001 3
schedule_1002 3
schedule_1003 3
schedule_1004 3
schedule_1005 3
schedule_1006 3
schedule_1007 3
schedule_1008 3
schedule_1009 3
stay_1000 25
stay_1001 26
stay_1002 27
stay_1003 28
stay_1004 29
stay_1005 30
stay_1006 31
stay_1007 32
stay_1008 33
stay_1009 34
sat
1010 40
Ulimate Cost 1010 2490
Num_timeslot 10 Optimal_cost 40
schedule_1010 3
schedule_1011 3
schedule_1012 3
schedule_1013 3
schedule_1014 3
schedule_1015 3
schedule_1016 3
schedule_1017 3
schedule_1018 3
schedule_1019 3
stay_1010 35
stay_1011 36
stay_1012 37
stay_1013 38
stay_1014 39
stay_1015 40
stay_1016 41
stay_1017 42
stay_1018 43
stay_1019 44
sat
1020 39
Ulimate Cost 1020 2529
Num_timeslot 10 Optimal_cost 39
schedule_1020 3
schedule_1021 3
schedule_1022 3
schedule_1023 4
schedule_1024 3
schedule_1025 3
schedule_1026 3
schedule_1027 3
schedule_1028 3
schedule_1029 3
stay_1020 45
stay_1021 46
stay_1022 47
stay_1023 1
stay_102

sat
1240 30
Ulimate Cost 1240 3392
Num_timeslot 10 Optimal_cost 30
schedule_1240 4
schedule_1241 4
schedule_1242 4
schedule_1243 4
schedule_1244 4
schedule_1245 4
schedule_1246 4
schedule_1247 4
schedule_1248 4
schedule_1249 4
stay_1240 4
stay_1241 5
stay_1242 6
stay_1243 7
stay_1244 8
stay_1245 9
stay_1246 10
stay_1247 11
stay_1248 12
stay_1249 13
sat
1250 30
Ulimate Cost 1250 3422
Num_timeslot 10 Optimal_cost 30
schedule_1250 4
schedule_1251 4
schedule_1252 4
schedule_1253 4
schedule_1254 4
schedule_1255 4
schedule_1256 4
schedule_1257 4
schedule_1258 4
schedule_1259 4
stay_1250 14
stay_1251 15
stay_1252 16
stay_1253 17
stay_1254 18
stay_1255 19
stay_1256 20
stay_1257 21
stay_1258 22
stay_1259 23
sat
1260 30
Ulimate Cost 1260 3452
Num_timeslot 10 Optimal_cost 30
schedule_1260 4
schedule_1261 4
schedule_1262 4
schedule_1263 4
schedule_1264 4
schedule_1265 4
schedule_1266 4
schedule_1267 4
schedule_1268 4
schedule_1269 4
stay_1260 24
stay_1261 25
stay_1262 26
stay_1263 27
stay_1264 28


# House-A Occupant-1 Greedy Schedule

In [15]:
greedy_schedule_house_A_occ_1 = greedy_schedule(clusters_house_A_occ_1, 1440)
pd.DataFrame(greedy_schedule_house_A_occ_1, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/DBSCAN_House-A_Occ-1_Greedy_Schedule.csv", index = False)

Here 0 18
selected zone 3
Here 18 22
selected zone 3
Here 22 148
selected zone 2
Here 148 161
selected zone 4
Here 161 169
selected zone 4
Here 169 654
selected zone 1
Here 654 735
selected zone 3
Here 735 796
selected zone 3
Here 796 819
selected zone 3
Here 819 900
selected zone 2
Here 900 930
selected zone 3
Here 930 973
selected zone 3
Here 973 1023
selected zone 3
Here 1023 1082
selected zone 3
Here 1082 1150
selected zone 3
Here 1150 1230
selected zone 3
Here 1230 1304
selected zone 3
Here 1304 1356
selected zone 3
Here 1356 1392
selected zone 3
Here 1392 1417
selected zone 3
Here 1417 1431
selected zone 3
Here 1431 1436
selected zone 3
Here 1436 1438
selected zone 3
Critical 1438
Critical 1439


# House-A Occupant-2 Attack Schedule

In [16]:
schedule_house_A_occ_2 = attack_scheduling(clusters_house_A_occ_2, num_timeslots, start_time, final_time)
pd.DataFrame(schedule_house_A_occ_2, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/DBSCAN_House-A_Occ-2_SHATTER_Schedule.csv", index = False)

sat
0 32
Ulimate Cost 0 32
Num_timeslot 10 Optimal_cost 32
schedule_0 3
schedule_1 0
schedule_2 3
schedule_3 3
schedule_4 0
schedule_5 3
schedule_6 3
schedule_7 3
schedule_8 3
schedule_9 3
stay_0 1
stay_1 1
stay_2 1
stay_3 2
stay_4 1
stay_5 1
stay_6 2
stay_7 3
stay_8 4
stay_9 5
sat
10 38
Ulimate Cost 10 70
Num_timeslot 10 Optimal_cost 38
schedule_10 4
schedule_11 4
schedule_12 3
schedule_13 3
schedule_14 3
schedule_15 3
schedule_16 3
schedule_17 3
schedule_18 3
schedule_19 3
stay_10 1
stay_11 2
stay_12 1
stay_13 2
stay_14 3
stay_15 4
stay_16 5
stay_17 6
stay_18 7
stay_19 8
sat
20 38
Ulimate Cost 20 108
Num_timeslot 10 Optimal_cost 38
schedule_20 4
schedule_21 4
schedule_22 3
schedule_23 3
schedule_24 3
schedule_25 3
schedule_26 3
schedule_27 3
schedule_28 3
schedule_29 3
stay_20 1
stay_21 2
stay_22 1
stay_23 2
stay_24 3
stay_25 4
stay_26 5
stay_27 6
stay_28 7
stay_29 8
sat
30 40
Ulimate Cost 30 148
Num_timeslot 10 Optimal_cost 40
schedule_30 3
schedule_31 3
schedule_32 3
schedule_33 3


sat
260 10
Ulimate Cost 260 752
Num_timeslot 10 Optimal_cost 10
schedule_260 1
schedule_261 1
schedule_262 1
schedule_263 1
schedule_264 1
schedule_265 1
schedule_266 1
schedule_267 1
schedule_268 1
schedule_269 1
stay_260 87
stay_261 88
stay_262 89
stay_263 90
stay_264 91
stay_265 92
stay_266 93
stay_267 94
stay_268 95
stay_269 96
sat
270 10
Ulimate Cost 270 762
Num_timeslot 10 Optimal_cost 10
schedule_270 1
schedule_271 1
schedule_272 1
schedule_273 1
schedule_274 1
schedule_275 1
schedule_276 1
schedule_277 1
schedule_278 1
schedule_279 1
stay_270 97
stay_271 98
stay_272 99
stay_273 100
stay_274 101
stay_275 102
stay_276 103
stay_277 104
stay_278 105
stay_279 106
sat
280 10
Ulimate Cost 280 772
Num_timeslot 10 Optimal_cost 10
schedule_280 1
schedule_281 1
schedule_282 1
schedule_283 1
schedule_284 1
schedule_285 1
schedule_286 1
schedule_287 1
schedule_288 1
schedule_289 1
stay_280 107
stay_281 108
stay_282 109
stay_283 110
stay_284 111
stay_285 112
stay_286 113
stay_287 114
stay_28

sat
500 30
Ulimate Cost 500 1044
Num_timeslot 10 Optimal_cost 30
schedule_500 4
schedule_501 4
schedule_502 4
schedule_503 4
schedule_504 4
schedule_505 4
schedule_506 4
schedule_507 4
schedule_508 4
schedule_509 4
stay_500 17
stay_501 18
stay_502 19
stay_503 20
stay_504 21
stay_505 22
stay_506 23
stay_507 24
stay_508 25
stay_509 26
sat
510 30
Ulimate Cost 510 1074
Num_timeslot 10 Optimal_cost 30
schedule_510 4
schedule_511 4
schedule_512 4
schedule_513 4
schedule_514 4
schedule_515 4
schedule_516 4
schedule_517 4
schedule_518 4
schedule_519 4
stay_510 27
stay_511 28
stay_512 29
stay_513 30
stay_514 31
stay_515 32
stay_516 33
stay_517 34
stay_518 35
stay_519 36
sat
520 12
Ulimate Cost 520 1086
Num_timeslot 10 Optimal_cost 12
schedule_520 4
schedule_521 1
schedule_522 1
schedule_523 1
schedule_524 1
schedule_525 1
schedule_526 1
schedule_527 1
schedule_528 1
schedule_529 1
stay_520 37
stay_521 1
stay_522 2
stay_523 3
stay_524 4
stay_525 5
stay_526 6
stay_527 7
stay_528 8
stay_529 9
sat


sat
750 0
Ulimate Cost 750 1325
Num_timeslot 10 Optimal_cost 0
schedule_750 0
schedule_751 0
schedule_752 0
schedule_753 0
schedule_754 0
schedule_755 0
schedule_756 0
schedule_757 0
schedule_758 0
schedule_759 0
stay_750 135
stay_751 136
stay_752 137
stay_753 138
stay_754 139
stay_755 140
stay_756 141
stay_757 142
stay_758 143
stay_759 144
sat
760 0
Ulimate Cost 760 1325
Num_timeslot 10 Optimal_cost 0
schedule_760 0
schedule_761 0
schedule_762 0
schedule_763 0
schedule_764 0
schedule_765 0
schedule_766 0
schedule_767 0
schedule_768 0
schedule_769 0
stay_760 145
stay_761 146
stay_762 147
stay_763 148
stay_764 149
stay_765 150
stay_766 151
stay_767 152
stay_768 153
stay_769 154
sat
770 0
Ulimate Cost 770 1325
Num_timeslot 10 Optimal_cost 0
schedule_770 0
schedule_771 0
schedule_772 0
schedule_773 0
schedule_774 0
schedule_775 0
schedule_776 0
schedule_777 0
schedule_778 0
schedule_779 0
stay_770 155
stay_771 156
stay_772 157
stay_773 158
stay_774 159
stay_775 160
stay_776 161
stay_777 1

sat
1000 20
Ulimate Cost 1000 1489
Num_timeslot 10 Optimal_cost 20
schedule_1000 2
schedule_1001 2
schedule_1002 2
schedule_1003 2
schedule_1004 2
schedule_1005 2
schedule_1006 2
schedule_1007 2
schedule_1008 2
schedule_1009 2
stay_1000 5
stay_1001 6
stay_1002 7
stay_1003 8
stay_1004 9
stay_1005 10
stay_1006 11
stay_1007 12
stay_1008 13
stay_1009 14
sat
1010 18
Ulimate Cost 1010 1507
Num_timeslot 10 Optimal_cost 18
schedule_1010 2
schedule_1011 2
schedule_1012 2
schedule_1013 2
schedule_1014 0
schedule_1015 2
schedule_1016 2
schedule_1017 2
schedule_1018 2
schedule_1019 2
stay_1010 15
stay_1011 16
stay_1012 17
stay_1013 18
stay_1014 1
stay_1015 1
stay_1016 2
stay_1017 3
stay_1018 4
stay_1019 5
sat
1020 20
Ulimate Cost 1020 1527
Num_timeslot 10 Optimal_cost 20
schedule_1020 2
schedule_1021 2
schedule_1022 2
schedule_1023 2
schedule_1024 2
schedule_1025 2
schedule_1026 2
schedule_1027 2
schedule_1028 2
schedule_1029 2
stay_1020 6
stay_1021 7
stay_1022 8
stay_1023 9
stay_1024 10
stay_1025

sat
1240 40
Ulimate Cost 1240 2167
Num_timeslot 10 Optimal_cost 40
schedule_1240 3
schedule_1241 3
schedule_1242 3
schedule_1243 3
schedule_1244 3
schedule_1245 3
schedule_1246 3
schedule_1247 3
schedule_1248 3
schedule_1249 3
stay_1240 37
stay_1241 38
stay_1242 39
stay_1243 40
stay_1244 41
stay_1245 42
stay_1246 43
stay_1247 44
stay_1248 45
stay_1249 46
sat
1250 40
Ulimate Cost 1250 2207
Num_timeslot 10 Optimal_cost 40
schedule_1250 3
schedule_1251 3
schedule_1252 3
schedule_1253 3
schedule_1254 3
schedule_1255 3
schedule_1256 3
schedule_1257 3
schedule_1258 3
schedule_1259 3
stay_1250 47
stay_1251 48
stay_1252 49
stay_1253 50
stay_1254 51
stay_1255 52
stay_1256 53
stay_1257 54
stay_1258 55
stay_1259 56
sat
1260 40
Ulimate Cost 1260 2247
Num_timeslot 10 Optimal_cost 40
schedule_1260 3
schedule_1261 3
schedule_1262 3
schedule_1263 3
schedule_1264 3
schedule_1265 3
schedule_1266 3
schedule_1267 3
schedule_1268 3
schedule_1269 3
stay_1260 57
stay_1261 58
stay_1262 59
stay_1263 60
stay_12

# House-A Occupant-2 Greedy Schedule

In [17]:
greedy_schedule_house_A_occ_2 = greedy_schedule(clusters_house_A_occ_2, 1440)
pd.DataFrame(greedy_schedule_house_A_occ_2, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/DBSCAN_House-A_Occ-2_Greedy_Schedule.csv", index = False)

Here 0 1
selected zone 3
Here 2 45
selected zone 2
Here 45 81
selected zone 3
Here 81 119
selected zone 3
Here 119 160
selected zone 3
Here 160 179
selected zone 4
Here 179 498
selected zone 1
Here 498 530
selected zone 4
Here 530 541
selected zone 1
Here 541 571
selected zone 4
Here 571 597
selected zone 4
Here 597 619
selected zone 4
Critical 619
Critical 620
Critical 621
Critical 622
Critical 623
Critical 624
Critical 625
Critical 626
Critical 627
Critical 628
Critical 629
Critical 630
Critical 631
Critical 632
Critical 633
Critical 634
Critical 635
Critical 636
Critical 637
Critical 638
Critical 639
Critical 640
Critical 641
Critical 642
Critical 643
Critical 644
Critical 645
Critical 646
Critical 647
Critical 648
Critical 649
Critical 650
Critical 651
Critical 652
Critical 653
Critical 654
Critical 655
Critical 656
Critical 657
Critical 658
Critical 659
Critical 660
Critical 661
Critical 662
Critical 663
Critical 664
Critical 665
Critical 666
Critical 667
Critical 668
Critical 669

# House-B Occupant-1 Attack Schedule

In [18]:
schedule_house_B_occ_1 = attack_scheduling(clusters_house_B_occ_1, num_timeslots, start_time, final_time)
pd.DataFrame(schedule_house_B_occ_1, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/DBSCAN_House-B_Occ-1_SHATTER_Schedule.csv", index = False)

sat
0 23
Ulimate Cost 0 23
Num_timeslot 10 Optimal_cost 23
schedule_0 2
schedule_1 2
schedule_2 2
schedule_3 2
schedule_4 2
schedule_5 2
schedule_6 4
schedule_7 4
schedule_8 4
schedule_9 2
stay_0 1
stay_1 2
stay_2 3
stay_3 4
stay_4 5
stay_5 6
stay_6 1
stay_7 2
stay_8 3
stay_9 1
sat
10 26
Ulimate Cost 10 49
Num_timeslot 10 Optimal_cost 26
schedule_10 2
schedule_11 2
schedule_12 4
schedule_13 4
schedule_14 4
schedule_15 2
schedule_16 2
schedule_17 4
schedule_18 4
schedule_19 4
stay_10 2
stay_11 3
stay_12 1
stay_13 2
stay_14 3
stay_15 1
stay_16 2
stay_17 1
stay_18 2
stay_19 3
sat
20 26
Ulimate Cost 20 75
Num_timeslot 10 Optimal_cost 26
schedule_20 2
schedule_21 2
schedule_22 4
schedule_23 4
schedule_24 4
schedule_25 2
schedule_26 2
schedule_27 4
schedule_28 4
schedule_29 4
stay_20 1
stay_21 2
stay_22 1
stay_23 2
stay_24 3
stay_25 1
stay_26 2
stay_27 1
stay_28 2
stay_29 3
sat
30 26
Ulimate Cost 30 101
Num_timeslot 10 Optimal_cost 26
schedule_30 2
schedule_31 2
schedule_32 4
schedule_33 4
s

sat
260 10
Ulimate Cost 260 405
Num_timeslot 10 Optimal_cost 10
schedule_260 1
schedule_261 1
schedule_262 1
schedule_263 1
schedule_264 1
schedule_265 1
schedule_266 1
schedule_267 1
schedule_268 1
schedule_269 1
stay_260 162
stay_261 163
stay_262 164
stay_263 165
stay_264 166
stay_265 167
stay_266 168
stay_267 169
stay_268 170
stay_269 171
sat
270 10
Ulimate Cost 270 415
Num_timeslot 10 Optimal_cost 10
schedule_270 1
schedule_271 1
schedule_272 1
schedule_273 1
schedule_274 1
schedule_275 1
schedule_276 1
schedule_277 1
schedule_278 1
schedule_279 1
stay_270 172
stay_271 173
stay_272 174
stay_273 175
stay_274 176
stay_275 177
stay_276 178
stay_277 179
stay_278 180
stay_279 181
sat
280 10
Ulimate Cost 280 425
Num_timeslot 10 Optimal_cost 10
schedule_280 1
schedule_281 1
schedule_282 1
schedule_283 1
schedule_284 1
schedule_285 1
schedule_286 1
schedule_287 1
schedule_288 1
schedule_289 1
stay_280 182
stay_281 183
stay_282 184
stay_283 185
stay_284 186
stay_285 187
stay_286 188
stay_28

sat
500 10
Ulimate Cost 500 645
Num_timeslot 10 Optimal_cost 10
schedule_500 1
schedule_501 1
schedule_502 1
schedule_503 1
schedule_504 1
schedule_505 1
schedule_506 1
schedule_507 1
schedule_508 1
schedule_509 1
stay_500 402
stay_501 403
stay_502 404
stay_503 405
stay_504 406
stay_505 407
stay_506 408
stay_507 409
stay_508 410
stay_509 411
sat
510 10
Ulimate Cost 510 655
Num_timeslot 10 Optimal_cost 10
schedule_510 1
schedule_511 1
schedule_512 1
schedule_513 1
schedule_514 1
schedule_515 1
schedule_516 1
schedule_517 1
schedule_518 1
schedule_519 1
stay_510 412
stay_511 413
stay_512 414
stay_513 415
stay_514 416
stay_515 417
stay_516 418
stay_517 419
stay_518 420
stay_519 421
sat
520 10
Ulimate Cost 520 665
Num_timeslot 10 Optimal_cost 10
schedule_520 1
schedule_521 1
schedule_522 1
schedule_523 1
schedule_524 1
schedule_525 1
schedule_526 1
schedule_527 1
schedule_528 1
schedule_529 1
stay_520 422
stay_521 423
stay_522 424
stay_523 425
stay_524 426
stay_525 427
stay_526 428
stay_52

sat
750 18
Ulimate Cost 750 1359
Num_timeslot 10 Optimal_cost 18
schedule_750 0
schedule_751 2
schedule_752 2
schedule_753 2
schedule_754 2
schedule_755 2
schedule_756 2
schedule_757 2
schedule_758 2
schedule_759 2
stay_750 2
stay_751 1
stay_752 2
stay_753 3
stay_754 4
stay_755 5
stay_756 6
stay_757 7
stay_758 8
stay_759 9
sat
760 20
Ulimate Cost 760 1379
Num_timeslot 10 Optimal_cost 20
schedule_760 2
schedule_761 2
schedule_762 2
schedule_763 2
schedule_764 2
schedule_765 2
schedule_766 2
schedule_767 2
schedule_768 2
schedule_769 2
stay_760 10
stay_761 11
stay_762 12
stay_763 13
stay_764 14
stay_765 15
stay_766 16
stay_767 17
stay_768 18
stay_769 19
sat
770 20
Ulimate Cost 770 1399
Num_timeslot 10 Optimal_cost 20
schedule_770 2
schedule_771 2
schedule_772 2
schedule_773 2
schedule_774 2
schedule_775 2
schedule_776 2
schedule_777 2
schedule_778 2
schedule_779 2
stay_770 20
stay_771 21
stay_772 22
stay_773 23
stay_774 24
stay_775 25
stay_776 26
stay_777 27
stay_778 28
stay_779 29
sat
7

sat
1000 27
Ulimate Cost 1000 1922
Num_timeslot 10 Optimal_cost 27
schedule_1000 4
schedule_1001 0
schedule_1002 4
schedule_1003 4
schedule_1004 4
schedule_1005 4
schedule_1006 4
schedule_1007 4
schedule_1008 4
schedule_1009 4
stay_1000 9
stay_1001 1
stay_1002 1
stay_1003 2
stay_1004 3
stay_1005 4
stay_1006 5
stay_1007 6
stay_1008 7
stay_1009 8
sat
1010 6
Ulimate Cost 1010 1928
Num_timeslot 10 Optimal_cost 6
schedule_1010 4
schedule_1011 4
schedule_1012 0
schedule_1013 0
schedule_1014 0
schedule_1015 0
schedule_1016 0
schedule_1017 0
schedule_1018 0
schedule_1019 0
stay_1010 9
stay_1011 10
stay_1012 1
stay_1013 2
stay_1014 3
stay_1015 4
stay_1016 5
stay_1017 6
stay_1018 7
stay_1019 8
sat
1020 0
Ulimate Cost 1020 1928
Num_timeslot 10 Optimal_cost 0
schedule_1020 0
schedule_1021 0
schedule_1022 0
schedule_1023 0
schedule_1024 0
schedule_1025 0
schedule_1026 0
schedule_1027 0
schedule_1028 0
schedule_1029 0
stay_1020 9
stay_1021 10
stay_1022 11
stay_1023 12
stay_1024 13
stay_1025 14
stay_

sat
1240 36
Ulimate Cost 1240 2547
Num_timeslot 10 Optimal_cost 36
schedule_1240 0
schedule_1241 3
schedule_1242 3
schedule_1243 3
schedule_1244 3
schedule_1245 3
schedule_1246 3
schedule_1247 3
schedule_1248 3
schedule_1249 3
stay_1240 1
stay_1241 1
stay_1242 2
stay_1243 3
stay_1244 4
stay_1245 5
stay_1246 6
stay_1247 7
stay_1248 8
stay_1249 9
sat
1250 22
Ulimate Cost 1250 2569
Num_timeslot 10 Optimal_cost 22
schedule_1250 3
schedule_1251 2
schedule_1252 2
schedule_1253 2
schedule_1254 2
schedule_1255 2
schedule_1256 2
schedule_1257 2
schedule_1258 2
schedule_1259 2
stay_1250 10
stay_1251 1
stay_1252 2
stay_1253 3
stay_1254 4
stay_1255 5
stay_1256 6
stay_1257 7
stay_1258 8
stay_1259 9
sat
1260 20
Ulimate Cost 1260 2589
Num_timeslot 10 Optimal_cost 20
schedule_1260 2
schedule_1261 2
schedule_1262 2
schedule_1263 2
schedule_1264 2
schedule_1265 2
schedule_1266 2
schedule_1267 2
schedule_1268 2
schedule_1269 2
stay_1260 10
stay_1261 11
stay_1262 12
stay_1263 13
stay_1264 14
stay_1265 15


# House-B Occupant-1 Greedy Schedule

In [19]:
greedy_schedule_house_B_occ_1 = greedy_schedule(clusters_house_B_occ_1, 1440)
pd.DataFrame(greedy_schedule_house_B_occ_1, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/DBSCAN_House-B_Occ-1_Greedy_Schedule.csv", index = False)

Here 0 61
selected zone 2
Here 61 64
selected zone 4
Here 64 67
selected zone 4
Here 67 70
selected zone 4
Here 70 73
selected zone 4
Here 73 76
selected zone 4
Here 76 79
selected zone 4
Here 79 82
selected zone 4
Here 82 85
selected zone 4
Here 85 87
selected zone 4
Here 87 89
selected zone 4
Here 89 91
selected zone 4
Here 91 93
selected zone 4
Here 93 95
selected zone 4
Here 95 97
selected zone 4
Here 97 99
selected zone 4
Here 99 100
selected zone 4
Here 100 101
selected zone 4
Here 101 102
selected zone 4
Here 102 103
selected zone 4
Here 103 104
selected zone 4
Here 104 105
selected zone 4
Here 105 106
selected zone 4
Here 106 107
selected zone 4
Here 107 108
selected zone 4
Here 108 109
selected zone 4
Here 109 110
selected zone 4
Here 110 111
selected zone 4
Here 111 112
selected zone 4
Here 112 595
selected zone 1
Here 595 606
selected zone 4
Here 606 636
selected zone 3
Here 636 667
selected zone 3
Here 667 699
selected zone 3
Here 699 716
selected zone 3
Here 716 725
select

# House-B Occupant-2 Attack Schedule

In [20]:
schedule_house_B_occ_2 = attack_scheduling(clusters_house_B_occ_2, num_timeslots, start_time, final_time)
pd.DataFrame(schedule_house_B_occ_2, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/DBSCAN_House-B_Occ-2_SHATTER_Schedule.csv", index = False)

sat
0 0
Ulimate Cost 0 0
Num_timeslot 10 Optimal_cost 0
schedule_0 0
schedule_1 0
schedule_2 0
schedule_3 0
schedule_4 0
schedule_5 0
schedule_6 0
schedule_7 0
schedule_8 0
schedule_9 0
stay_0 1
stay_1 2
stay_2 3
stay_3 4
stay_4 5
stay_5 6
stay_6 7
stay_7 8
stay_8 9
stay_9 10
sat
10 0
Ulimate Cost 10 0
Num_timeslot 10 Optimal_cost 0
schedule_10 0
schedule_11 0
schedule_12 0
schedule_13 0
schedule_14 0
schedule_15 0
schedule_16 0
schedule_17 0
schedule_18 0
schedule_19 0
stay_10 11
stay_11 12
stay_12 13
stay_13 14
stay_14 15
stay_15 16
stay_16 17
stay_17 18
stay_18 19
stay_19 20
sat
20 12
Ulimate Cost 20 12
Num_timeslot 10 Optimal_cost 12
schedule_20 0
schedule_21 0
schedule_22 0
schedule_23 4
schedule_24 0
schedule_25 4
schedule_26 0
schedule_27 4
schedule_28 0
schedule_29 4
stay_20 21
stay_21 22
stay_22 23
stay_23 1
stay_24 1
stay_25 1
stay_26 1
stay_27 1
stay_28 1
stay_29 1
sat
30 15
Ulimate Cost 30 27
Num_timeslot 10 Optimal_cost 15
schedule_30 0
schedule_31 4
schedule_32 0
schedule

sat
260 0
Ulimate Cost 260 126
Num_timeslot 10 Optimal_cost 0
schedule_260 0
schedule_261 0
schedule_262 0
schedule_263 0
schedule_264 0
schedule_265 0
schedule_266 0
schedule_267 0
schedule_268 0
schedule_269 0
stay_260 12
stay_261 13
stay_262 14
stay_263 15
stay_264 16
stay_265 17
stay_266 18
stay_267 19
stay_268 20
stay_269 21
sat
270 0
Ulimate Cost 270 126
Num_timeslot 10 Optimal_cost 0
schedule_270 0
schedule_271 0
schedule_272 0
schedule_273 0
schedule_274 0
schedule_275 0
schedule_276 0
schedule_277 0
schedule_278 0
schedule_279 0
stay_270 22
stay_271 23
stay_272 24
stay_273 25
stay_274 26
stay_275 27
stay_276 28
stay_277 29
stay_278 30
stay_279 31
sat
280 0
Ulimate Cost 280 126
Num_timeslot 10 Optimal_cost 0
schedule_280 0
schedule_281 0
schedule_282 0
schedule_283 0
schedule_284 0
schedule_285 0
schedule_286 0
schedule_287 0
schedule_288 0
schedule_289 0
stay_280 32
stay_281 33
stay_282 34
stay_283 35
stay_284 36
stay_285 37
stay_286 38
stay_287 39
stay_288 40
stay_289 41
sat


sat
510 10
Ulimate Cost 510 315
Num_timeslot 10 Optimal_cost 10
schedule_510 1
schedule_511 1
schedule_512 1
schedule_513 1
schedule_514 1
schedule_515 1
schedule_516 1
schedule_517 1
schedule_518 1
schedule_519 1
stay_510 102
stay_511 103
stay_512 104
stay_513 105
stay_514 106
stay_515 107
stay_516 108
stay_517 109
stay_518 110
stay_519 111
sat
520 10
Ulimate Cost 520 325
Num_timeslot 10 Optimal_cost 10
schedule_520 1
schedule_521 1
schedule_522 1
schedule_523 1
schedule_524 1
schedule_525 1
schedule_526 1
schedule_527 1
schedule_528 1
schedule_529 1
stay_520 112
stay_521 113
stay_522 114
stay_523 115
stay_524 116
stay_525 117
stay_526 118
stay_527 119
stay_528 120
stay_529 121
sat
530 27
Ulimate Cost 530 352
Num_timeslot 10 Optimal_cost 27
schedule_530 1
schedule_531 1
schedule_532 1
schedule_533 4
schedule_534 4
schedule_535 4
schedule_536 4
schedule_537 3
schedule_538 3
schedule_539 3
stay_530 122
stay_531 123
stay_532 124
stay_533 1
stay_534 2
stay_535 3
stay_536 4
stay_537 1
stay

sat
760 30
Ulimate Cost 760 1115
Num_timeslot 10 Optimal_cost 30
schedule_760 4
schedule_761 4
schedule_762 4
schedule_763 4
schedule_764 4
schedule_765 4
schedule_766 4
schedule_767 4
schedule_768 4
schedule_769 4
stay_760 6
stay_761 7
stay_762 8
stay_763 9
stay_764 10
stay_765 11
stay_766 12
stay_767 13
stay_768 14
stay_769 15
sat
770 3
Ulimate Cost 770 1118
Num_timeslot 10 Optimal_cost 3
schedule_770 4
schedule_771 0
schedule_772 0
schedule_773 0
schedule_774 0
schedule_775 0
schedule_776 0
schedule_777 0
schedule_778 0
schedule_779 0
stay_770 16
stay_771 1
stay_772 2
stay_773 3
stay_774 4
stay_775 5
stay_776 6
stay_777 7
stay_778 8
stay_779 9
sat
780 0
Ulimate Cost 780 1118
Num_timeslot 10 Optimal_cost 0
schedule_780 0
schedule_781 0
schedule_782 0
schedule_783 0
schedule_784 0
schedule_785 0
schedule_786 0
schedule_787 0
schedule_788 0
schedule_789 0
stay_780 10
stay_781 11
stay_782 12
stay_783 13
stay_784 14
stay_785 15
stay_786 16
stay_787 17
stay_788 18
stay_789 19
sat
790 0
Ul

sat
1010 0
Ulimate Cost 1010 1118
Num_timeslot 10 Optimal_cost 0
schedule_1010 0
schedule_1011 0
schedule_1012 0
schedule_1013 0
schedule_1014 0
schedule_1015 0
schedule_1016 0
schedule_1017 0
schedule_1018 0
schedule_1019 0
stay_1010 240
stay_1011 241
stay_1012 242
stay_1013 243
stay_1014 244
stay_1015 245
stay_1016 246
stay_1017 247
stay_1018 248
stay_1019 249
sat
1020 0
Ulimate Cost 1020 1118
Num_timeslot 10 Optimal_cost 0
schedule_1020 0
schedule_1021 0
schedule_1022 0
schedule_1023 0
schedule_1024 0
schedule_1025 0
schedule_1026 0
schedule_1027 0
schedule_1028 0
schedule_1029 0
stay_1020 250
stay_1021 251
stay_1022 252
stay_1023 253
stay_1024 254
stay_1025 255
stay_1026 256
stay_1027 257
stay_1028 258
stay_1029 259
sat
1030 0
Ulimate Cost 1030 1118
Num_timeslot 10 Optimal_cost 0
schedule_1030 0
schedule_1031 0
schedule_1032 0
schedule_1033 0
schedule_1034 0
schedule_1035 0
schedule_1036 0
schedule_1037 0
schedule_1038 0
schedule_1039 0
stay_1030 260
stay_1031 261
stay_1032 262
sta

sat
1240 0
Ulimate Cost 1240 1164
Num_timeslot 10 Optimal_cost 0
schedule_1240 0
schedule_1241 0
schedule_1242 0
schedule_1243 0
schedule_1244 0
schedule_1245 0
schedule_1246 0
schedule_1247 0
schedule_1248 0
schedule_1249 0
stay_1240 1
stay_1241 2
stay_1242 3
stay_1243 4
stay_1244 5
stay_1245 6
stay_1246 7
stay_1247 8
stay_1248 9
stay_1249 10
sat
1250 0
Ulimate Cost 1250 1164
Num_timeslot 10 Optimal_cost 0
schedule_1250 0
schedule_1251 0
schedule_1252 0
schedule_1253 0
schedule_1254 0
schedule_1255 0
schedule_1256 0
schedule_1257 0
schedule_1258 0
schedule_1259 0
stay_1250 11
stay_1251 12
stay_1252 13
stay_1253 14
stay_1254 15
stay_1255 16
stay_1256 17
stay_1257 18
stay_1258 19
stay_1259 20
sat
1260 0
Ulimate Cost 1260 1164
Num_timeslot 10 Optimal_cost 0
schedule_1260 0
schedule_1261 0
schedule_1262 0
schedule_1263 0
schedule_1264 0
schedule_1265 0
schedule_1266 0
schedule_1267 0
schedule_1268 0
schedule_1269 0
stay_1260 21
stay_1261 22
stay_1262 23
stay_1263 24
stay_1264 25
stay_1265

# House-B Occupant-2 Greedy Schedule

In [21]:
greedy_schedule_house_B_occ_2 = greedy_schedule(clusters_house_B_occ_2, 1440)
pd.DataFrame(greedy_schedule_house_B_occ_2, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/DBSCAN_House-B_Occ-2_Greedy_Schedule.csv", index = False)

Critical 0
Critical 1
Critical 2
Critical 3
Critical 4
Critical 5
Critical 6
Critical 7
Critical 8
Critical 9
Critical 10
Critical 11
Critical 12
Critical 13
Critical 14
Critical 15
Critical 16
Critical 17
Critical 18
Critical 19
Critical 20
Critical 21
Here 22 23
selected zone 4
Here 23 24
selected zone 4
Here 24 25
selected zone 4
Here 25 26
selected zone 4
Here 26 27
selected zone 4
Here 27 28
selected zone 4
Here 28 29
selected zone 4
Here 29 30
selected zone 4
Here 30 31
selected zone 4
Here 31 32
selected zone 4
Here 32 33
selected zone 4
Here 33 34
selected zone 4
Here 34 35
selected zone 4
Here 35 37
selected zone 4
Here 37 38
selected zone 4
Here 38 39
selected zone 4
Here 39 40
selected zone 4
Critical 40
Critical 41
Critical 42
Critical 43
Critical 44
Critical 45
Critical 46
Critical 47
Critical 48
Critical 49
Critical 50
Critical 51
Critical 52
Critical 53
Critical 54
Critical 55
Critical 56
Critical 57
Critical 58
Critical 59
Critical 60
Critical 61
Critical 62
Critical 63